In [1]:
import pandas as pd
import plotly.express as px
import requests
from semanticscholar import SemanticScholar

from paper_analysis import semantic_scholar as s2

In [2]:
paper_id = s2.get_id_from_url(
    "https://www.semanticscholar.org/paper/Mapping-poverty-at-multiple-geographical-scales-Nicolò-Fabrizi/f24dc845602d5b395b3c33697e6edcf29ad5776a"
)

In [3]:
paper = s2.get_paper_from_id(paper_id=paper_id)

In [2]:
sch = SemanticScholar()
paper = sch.get_paper("f24dc845602d5b395b3c33697e6edcf29ad5776a")
paper.title

'Mapping poverty at multiple geographical scales'

In [4]:
def get_references(references: list) -> list:
    data = []
    for ref in references:
        if ref["publicationVenue"]:
            if ref["publicationVenue"]["type"]:
                publication_venue_type = ref["publicationVenue"]["type"]
        else:
            publication_venue_type = None
        data.append(
            (
                ref["paperId"],
                ref["title"],
                ref["url"],
                ref["publicationDate"],
                ref["venue"],
                publication_venue_type,
                ref["referenceCount"],
                ref["citationCount"],
            )
        )
    return data


print([k for k in paper.references[0].keys()])
data = get_references(paper.references)
df = pd.DataFrame(
    data,
    columns=[
        "paperId",
        "title",
        "url",
        "publicationDate",
        "venue",
        "publication_venue_type",
        "referenceCount",
        "citationCount",
    ],
)
df.citationCount = df.citationCount.fillna(0)
df["binning"] = pd.qcut(df.citationCount, 8, labels=False)
df["binning"] = df["binning"] + 1
# df['binning'] = pd.cut(df.citationCount, bins=4, labels=False)
df.head()

['paperId', 'externalIds', 'corpusId', 'publicationVenue', 'url', 'title', 'abstract', 'venue', 'year', 'referenceCount', 'citationCount', 'influentialCitationCount', 'isOpenAccess', 'openAccessPdf', 'fieldsOfStudy', 's2FieldsOfStudy', 'publicationTypes', 'publicationDate', 'journal', 'citationStyles', 'authors']


,paperId,title,url,publicationDate,venue,publication_venue_type,referenceCount,citationCount,binning
0,90405063133184d0ad29b426feb490ece0ce83b9,A review of machine learning and satellite ima...,https://www.semanticscholar.org/paper/90405063...,2023-02-01,Journal of International Development,journal,37.0,2.0,1
1,830d29d57e9c54bff7803aa52b00dc56535b4470,Multivariate mixture model for small area esti...,https://www.semanticscholar.org/paper/830d29d5...,2022-12-01,Journal of the Royal Statistical Society: Seri...,None,27.0,1.0,1
2,d253b41297e7ec43cec7f02401196125188e0824,Guidelines to Small Area Estimation for Povert...,https://www.semanticscholar.org/paper/d253b412...,2022-06-16,,None,0.0,2.0,1
3,2c36a2514c98f47483a40c5152b337a2bc4450c0,A Computationally Efficient Approach to Fully ...,https://www.semanticscholar.org/paper/2c36a251...,2022-03-23,,None,0.0,1.0,1
4,3bba63cd7b14cdcd458a4fa6be8c78d2582cd65c,Accounting for survey design in Bayesian disag...,https://www.semanticscholar.org/paper/3bba63cd...,2021-12-13,Annals of Applied Statistics,journal,51.0,2.0,1


In [30]:
fig = px.scatter(
    df,
    x="publicationDate",
    y="venue",
    # size="binning",
    size="citationCount",
    size_max=60,
    # size_min=20,
    color="citationCount",
    # marginal_x="histogram",
    title=f"Paper: {paper.title} ({paper.year})",
    hover_data=["title", "referenceCount", "citationCount"],
    # width=1000,
    height=int(25 * len(df.venue.unique())),
    # height=700,
)

fig.layout.yaxis.type = "category"
fig.update_yaxes(dtick=1)
fig.show()

In [2]:
paper_url = "https://arxiv.org/abs/2305.19513"
endpoint = "https://api.semanticscholar.org/graph/v1/paper/"
url = endpoint + "URL:" + paper_url
print(url)

https://api.semanticscholar.org/graph/v1/paper/URL:https://arxiv.org/abs/2305.19513


In [3]:
SUPPORTED_WEBSITE_URL = [
    "semanticscholar.org",
    "arxiv.org",
    "aclweb.org",
    "acm.org",
    "biorxiv.org",
]


def generate_url(input_str: str) -> str:
    endpoint = "https://api.semanticscholar.org/graph/v1/paper/"
    if any([x for x in SUPPORTED_WEBSITE_URL if x in input_str]):
        return endpoint + "URL:" + paper_url
    else:
        print(f"Paper {paper_url} not found")


def get_paper_id(paper_url: str):
    s2_url = generate_url(paper_url)
    print(s2_url)
    response = requests.get(s2_url)
    return response.json()["paperId"]

In [4]:
paper_id = get_paper_id(paper_url)

https://api.semanticscholar.org/graph/v1/paper/URL:https://arxiv.org/abs/2305.19513


In [6]:
paper = s2.get_paper_from_id(paper_id=paper_id)
df_references = s2.get_references_df(paper.references)
title = f"Paper: {paper.title} ({paper.year})"
fig = s2.plot_references_timeline(df_references, title=title)
fig.show()